In [20]:
import tensorflow
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [7]:
class splitData():
    def __init__(self):
        self.X_train = []
        self.X_test = []
        self.y_train = []
        self.y_test = []
    
    def split(self, x, y):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(x, y, test_size=0.1, random_state=42)

In [21]:
features = ['district_encoder',\
        'area',\
        'new_num_floors',\
        'new_bedrooms',\
        'houseTypes_Bán Luxury home',\
        'houseTypes_Bán Nhà',\
        'houseTypes_Bán Nhà cổ',\
        'houseTypes_Bán Nhà mặt phố',\
        'houseTypes_Bán Nhà riêng']

df = pd.read_excel('HCM_data.xlsx')
df_tranform = pd.DataFrame(data = StandardScaler().fit_transform(df.loc[:, features].values), columns = features)
y = df['price'].values
x = df_tranform[features].values
data = splitData()
data.split(x, y)

In [30]:

class ModelRNN():
    def __init__(self):
        self.model = Sequential()
        self.model.add(Dense(units=20, activation='relu')) #fully connected networks
        self.model.add(Dense(units=20, activation='relu'))
        self.model.add(Dense(units=20, activation='relu'))
        self.model.add(Dense(units=20, activation='relu'))
        self.model.add(Dense(units=1)) #output layer
        self.model.compile(optimizer='adam', loss='mse')
    
    def trainning(self, data):
        self.model.fit(data.X_train, data.y_train,
                validation_data=(data.X_test, data.y_test),
                batch_size = 128,
                epochs = 400)
        
    def evaluation(self, data):
        y_pred =  self.model.predict(data.X_test)
        from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score
        print("MSE: ",mean_squared_error(data.y_test, y_pred))
        print("MAE: ", mean_absolute_error(data.y_test, y_pred))
        np.sqrt(mean_squared_error(data.y_test, y_pred))
        print("variance: ", explained_variance_score(data.y_test, y_pred))

In [31]:
model_rnn = ModelRNN()
model_rnn.trainning(data)
model_rnn.evaluation(data)

Epoch 1/400
33/33 [==============================] - 2s 10ms/step - loss: 534.0795 - val_loss: 617.0577
Epoch 2/400
33/33 [==============================] - 0s 3ms/step - loss: 510.1194 - val_loss: 575.1830
Epoch 3/400
33/33 [==============================] - 0s 3ms/step - loss: 452.4164 - val_loss: 503.2667
Epoch 4/400
33/33 [==============================] - 0s 3ms/step - loss: 414.4634 - val_loss: 487.6142
Epoch 5/400
33/33 [==============================] - 0s 3ms/step - loss: 405.8899 - val_loss: 483.5196
Epoch 6/400
33/33 [==============================] - 0s 2ms/step - loss: 402.8477 - val_loss: 479.3450
Epoch 7/400
33/33 [==============================] - 0s 2ms/step - loss: 401.8125 - val_loss: 478.1667
Epoch 8/400
33/33 [==============================] - 0s 3ms/step - loss: 400.2846 - val_loss: 476.3878
Epoch 9/400
33/33 [==============================] - 0s 2ms/step - loss: 395.3492 - val_loss: 473.8303
Epoch 10/400
33/33 [==============================] - 0s 3ms/step - loss